## Align & crop faces

In [ ]:
import csv

In [ ]:
#note: add index: 9392, 14178, 20271, ...

filenames_skip = []
with open("utkface_filenames_skip.txt", "r") as f:
    reader = csv.reader(f,delimiter='\n')
    try:
        filenames_skip = [row[0] for row in reader]
    except csv.Error as e:
        print(e)

print(len(filenames_skip))
print(filenames_skip[-6:])

In [ ]:
from skimage import transform as trans
from mtcnn.mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt
from panya_s.utils import align_with_margin

import os

In [ ]:
img_path = "/home/krittametht/dataset/UTKFace_inthewild/"
out_path = "/home/krittametht/dataset/UTKFace_crop_m-4/"

filenames = os.listdir(img_path)
print('# files:', len(filenames))

fi_skip = [filenames.index(f) for f in filenames_skip]    
print('len(fi_skip):', len(fi_skip))

In [ ]:
return_box = False
# return_box = True
needPlot = False
# needPlot = True
needSkip = False

needSave = True

for fi in range(0,len(filenames)):
# for fi in fi_skip: ## for images with width > 1000

    if needSkip and fi in fi_skip:
        print('\n skip: index:', fi, 'filename:', filenames[fi])
        continue

    img_name = filenames[fi]
    img = cv2.imread(os.path.join(img_path, img_name))[...,::-1]
    
    height, width, depth = img.shape
    W = 1000.
    if width > 1000:
        imgScale = W/width
        newX,newY = img.shape[1]*imgScale, img.shape[0]*imgScale
        newimg = cv2.resize(img,(int(newX),int(newY)))
        img = newimg
    
    output = align_with_margin(img, output_image_size=227, m=0.4, return_box=return_box)
    if type(output) is str:
        print('\n', img_name, '\t', output)
    else:
        warped, newsrc, newbox = output
        if needSave:
            cv2.imwrite(os.path.join(out_path, img_name + "_m-4.jpg"), cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
#             print(os.path.join(out_path, img_name + "_m-4.jpg"))
        if needPlot and newbox is not None:
            plt.imshow(image_with_box_and_points(warped, newsrc.astype(int), highlight_color = (255,155,0), 
                                             box = [newbox[0,0],newbox[0,1],newbox[-1,0]-newbox[0,0],newbox[-1,1]-newbox[0,1]]))
    if fi % 10 == 0:    print(fi, end = '\t')